In [1]:
import ast 
import pandas as pd

df_usage = pd.read_csv('data/baseline_loc_time.txt', sep='\t')
df_usage['app_seq'] = df_usage['app_seq'].apply(ast.literal_eval)
df_usage['time_seq'] = df_usage['time_seq'].apply(ast.literal_eval)
df_usage['time'] = df_usage['time'].apply(lambda x: str(x))

In [2]:
df_usage.head()

,user,time,app,location,app_seq,time_seq
0,0,201604200816,612,8194,"[361, 361, 31, 360]","[3.0, 0.0, 0.0, 0.0]"
1,0,201604200817,31,8194,"[361, 31, 360, 612]","[0.0, 0.0, 0.0, 0.0]"
2,0,201604200817,360,8194,"[31, 360, 612, 31]","[1.0, 1.0, 1.0, 0.0]"
3,0,201604200817,361,8194,"[360, 612, 31, 360]","[1.0, 1.0, 0.0, 0.0]"
4,0,201604200824,1,8192,"[612, 31, 360, 361]","[1.0, 0.0, 0.0, 0.0]"


### 计算朴素贝叶斯的输入特征

In [4]:
import datetime

df_nb = df_usage.copy()

def prep_time(t):
    t = t[:-2]
    weekday = datetime.datetime.strptime(t[:-2], '%Y%m%d').weekday()
    if weekday >= 5:
        weekday = '1'
    else:
        weekday = '0'
    return '{}_{}'.format(weekday, t[-2:])

df_nb['time'] = df_nb['time'].apply(lambda x: prep_time(x))
df_nb['location'] = df_nb['location'].apply(lambda x:str(x))

app_seq_str = df_nb['app_seq'].apply(lambda app_seq: ' '.join([str(app) for app in app_seq]))
df_nb['nb_input'] = df_nb['time'] + '_' + df_nb['location'] + ' ' + app_seq_str

df_nb.head()

,user,time,app,location,app_seq,time_seq,nb_input
0,0,0_08,612,8194,"[361, 361, 31, 360]","[3.0, 0.0, 0.0, 0.0]",0_08_8194 361 361 31 360
1,0,0_08,31,8194,"[361, 31, 360, 612]","[0.0, 0.0, 0.0, 0.0]",0_08_8194 361 31 360 612
2,0,0_08,360,8194,"[31, 360, 612, 31]","[1.0, 1.0, 1.0, 0.0]",0_08_8194 31 360 612 31
3,0,0_08,361,8194,"[360, 612, 31, 360]","[1.0, 1.0, 0.0, 0.0]",0_08_8194 360 612 31 360
4,0,0_08,1,8192,"[612, 31, 360, 361]","[1.0, 0.0, 0.0, 0.0]",0_08_8192 612 31 360 361
